In [2]:
import os
import json

from io import BytesIO
from pathlib import Path

from urllib.parse import urlparse
from urllib.parse import parse_qs

import requests

import numpy as np
import pandas as pd

from PIL import Image
from jinja2 import Template

In [3]:
folder = Path('08-deep-learning')

os.makedirs(folder / 'images', exist_ok=True)

with open(folder / 'meta.json') as f_in:
    data = json.load(f_in)


session = data['session']
session_name = data['name']

In [4]:
df = pd.read_csv(folder / data['data'])
df = df.fillna('')
df

,lesson,name,page_name,video,slides,notebook
0,1,Fashion classification,01-fashion-classification.md,https://www.youtube.com/watch?v=it1Lu7NmMpw,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
1,2,TensorFlow and Keras,02-tensorflow-keras.md,https://www.youtube.com/watch?v=R6o_CUmoN9Q,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
2,3,Pre-trained convolutional neural networks,03-pretrained-models.md,https://www.youtube.com/watch?v=qGDXEz-cr6M,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
3,4,Convolutional neural networks,04-conv-neural-nets.md,https://www.youtube.com/watch?v=BN-fnYzbdc8,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
4,5,Transfer learning,05-transfer-learning.md,https://www.youtube.com/watch?v=WKHylqfNmq4,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
5,6,Adjusting the learning rate,06-learning-rate.md,https://www.youtube.com/watch?v=2gPmRRGz0Hc,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
6,7,Checkpointing,07-checkpointing.md,https://www.youtube.com/watch?v=NRpGUx0o3Ps,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
7,8,Adding more layers,08-more-layers.md,https://www.youtube.com/watch?v=bSRRrorvAZs,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
8,9,Regularization and dropout,09-dropout.md,https://www.youtube.com/watch?v=74YmhVM6FTM,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...
9,10,Data augmentation,10-augmentation.md,https://www.youtube.com/watch?v=aoPfVsS3BDE,https://www.slideshare.net/AlexeyGrigorev/ml-z...,https://github.com/alexeygrigorev/mlbookcamp-c...


In [11]:
template_string = """
## {{ session }}.{{ lesson }} {{ name }}

{% if video %}<a href="{{ video }}"><img src="{{ thumbnail }}"></a>{% endif %}
{% if coming_soon %}COMING SOON{% endif %} 

{% if slides %}[Slides]({{ slides }}){% endif %}


## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Navigation

{{ navigation }}
"""

In [6]:
def download_thumbnail(video, session, lesson):
    if type(lesson) in [np.int64]:
        thumbnail_file = 'thumbnail-%d-%02d.jpg' % (session, lesson)
    else:
        thumbnail_file = 'thumbnail-%d-%s.jpg' % (session, lesson)
    thumbnail_file_full = '%s/images/%s' % (folder, thumbnail_file)

    if os.path.exists(thumbnail_file_full):
        print(thumbnail_file_full, 'exists')
        return 'images/%s' % thumbnail_file

    video_id = parse_qs(urlparse(video).query)['v'][0]
    print('processing video', video_id)
    thumbnail_url = 'https://img.youtube.com/vi/%s/0.jpg' % video_id

    response = requests.get(thumbnail_url)
    thumbnail = Image.open(BytesIO(response.content))
    w_img, h_img = thumbnail.size

    play = Image.open('../images/play.png')
    w_play, h_play = play.size
    
    x0 = w_img // 2 - w_play // 2
    y0 = h_img // 2 - h_play // 2

    thumbnail.paste(play, (x0, y0), play)
    thumbnail.save(thumbnail_file_full, quality=90)

    print('saved to', thumbnail_file_full)

    return 'images/%s' % thumbnail_file

In [13]:
create_files = True

for i in range(len(df)):
    row = df.loc[i]
    params = row.to_dict()
    params['session'] = session

    navigation = [
        '* [Machine Learning Zoomcamp course](../)',
        '* [Session %s: %s](./)' % (session, session_name) 
    ]

    if i > 0:
        prev_row = df.loc[i - 1]
        line = '* Previous: [%s](%s)' % (prev_row['name'], prev_row.page_name)
        navigation.append(line)

    if i + 1 < len(df):
        next_row = df.loc[i + 1]
        line = '* Next: [%s](%s)' % (next_row['name'], next_row.page_name)
        navigation.append(line)
        
    params['navigation'] = '\n'.join(navigation)
    
    if row.video and row.video != 'NOT_READY':
        thumbnail = download_thumbnail(row.video, session, row.lesson)
        params['thumbnail'] = thumbnail

    if row.video == 'NOT_READY':
        del params['video']
        params['coming_soon'] = True
    
    if 'video' in params and len(params['video']) > 0:
        params['video'] = params['video'] + '&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR'

    template = Template(template_string)
    rendered = template.render(**params)
    print(rendered)
    
    page_full = '%s/%s' % (folder, row.page_name)
    
    print()

    if create_files:
        if (not os.path.exists(page_full)) or os.path.getsize(page_full) <= 1:
            with open(page_full, 'w', encoding='utf-8') as f_out:
                f_out.write(rendered)
            print("created", page_full)
        else:
            with open(page_full, 'a', encoding='utf-8') as f_out:
                f_out.write('\n\n========\n\n')
                f_out.write(rendered)
            print("APPENDED TO", page_full)
    
    print()
    print()

processing video 2viqmJ_NpgE
saved to 07-bentoml-production/images/thumbnail-7-01.jpg

## 7.1 Intro/Session Overview

<a href="https://www.youtube.com/watch?v=2viqmJ_NpgE&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR"><img src="images/thumbnail-7-01.jpg"></a>
 




## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Navigation

* [Machine Learning Zoomcamp course](../)
* [Session 7: Production-Ready Machine Learning (Bento ML)](./)
* Next: [Building Your Prediction Service with BentoML](02-build-bento-service.md)

created 07-bentoml-production/01-intro.md


processing video bWdEVlUw1CA
saved to 07-bentoml-production/images/thumbnail-7-02.jpg

## 7.2 Building Your Prediction Service with BentoML

<a href="https://www.youtube.com/watch?v=bWdEVlUw1CA&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR"><img

In [8]:
download_thumbnail('https://www.youtube.com/watch?v=WZCjsyV8hZE', 8, '01b')

processing video WZCjsyV8hZE
saved to 08-deep-learning/images/thumbnail-8-01b.jpg


'images/thumbnail-8-01b.jpg'